In [5]:
from datetime import datetime 

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/ run-{}/".format(root_logdir, now)

In [1]:
import tensorflow as tf
import numpy as np

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data")

n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_val = accuracy.eval(feed_dict={X:mnist.validation.images, y:mnist.validation.labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
0 Train accuracy: 0.84 Val accuracy: 0.9102
1 Train accuracy: 0.96 Val accuracy: 0.9314
2 Train accuracy: 0.94 Val accuracy: 0.9396
3 Train accuracy: 0.98 Val accuracy: 0.9496
4 Train accuracy: 0.98 Val accuracy: 0.9522
5 Train accuracy: 0.96 Val accuracy: 0.9562
6 Train accuracy: 0.96 Val accuracy: 0.9606
7 Train accuracy: 0.98 Val accuracy: 0.9622
8 Train accuracy: 0.98 Val accuracy: 0.9642
9 Train accuracy: 0.94 Val accuracy: 0.9666
10 Train accuracy: 0.98 Val accuracy: 0.9666
11 Train accuracy: 0.98 Val accuracy: 0.9698
12 Train accuracy: 1.0 Val accuracy: 0.9702
13 Train accuracy: 0.96 Val accuracy: 0.9702
14 Train accuracy: 0.98 Val accuracy: 0.9716
15 Train accuracy: 1.0 Val accuracy: 0.9724
16 Train accuracy: 0.98 Val accuracy: 0.974
17 Train accuracy: 1.0 Val accuracy: 0.9738
18 Train accur

In [33]:
# load the kaggle test now and see what we get
import os
import pandas as pd

MNIST_PATH = os.path.join("datasets", "mnist")

def load_mnist_dataset():
    csv_path = os.path.join(MNIST_PATH, "kaggle_mnist_test.csv")
    return pd.read_csv(csv_path)

X_train = load_mnist_dataset().as_matrix()

normalizer = lambda x: x / 256.0
X_train_scaled = normalizer(X_train)
y_pred = []

for i in range(len(X_train_scaled)):
    y_pred.append(0)

# just load the model and run
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    for i in range(len(X_train_scaled)):
        Z = logits.eval(feed_dict={X: [X_train_scaled[i]]})
        y_pred[i] = np.argmax(Z, axis=1)[0]

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [36]:
df = pd.DataFrame(data = {'ImageId':np.arange(1, len(X_train_scaled) + 1, 1), 'Label':y_pred})
df.to_csv('test.csv', index=False)